In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
!git clone https://github.com/NidhiAI/Speaker-Recognition-GMM

fatal: destination path 'Speaker-Recognition-GMM' already exists and is not an empty directory.


In [2]:
!pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=fd6fa85c6e6b53914f3bf70ebad8b76e8af76b3340f9f170f70c6cf401e19363
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python_speech_features


In [3]:
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc

def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
              first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines 
    delta to make it 40 dim feature vector"""    
    
    mfcc_feature = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature,delta)) 
    return combined

In [5]:
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
#from featureextraction import extract_features
import warnings
warnings.filterwarnings("ignore")

#path to training data
source   ="/kaggle/working/Speaker-Recognition-GMM/trainingData/"

#path to store trainged GMM models
dest = "/kaggle/working/Speaker-Recognition-GMM/speakerTrainedModelsGMM/"

#training data files names and folders  
train_file = "/kaggle/working/Speaker-Recognition-GMM/trainingDataPath.txt"
file_paths = open(train_file,'r')

count = 1
# Extracting features for each speaker (10 files per speakers)
features = np.asarray(())
for path in file_paths:    
    path = path.strip()   
    print (path)
    
    # read the audio
    sr,audio = read(source+path)
    
    # extract 40 dimensional MFCC & delta MFCC features
    vector   = extract_features(audio,sr)
    
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
    # when features of 5 files of speaker are concatenated, then do model training
	# -> if count == 5: --> edited below
    if count == 10:    
        gmm = GMM(n_components = 5, covariance_type='diag',n_init = 3)
        gmm.fit(features)
        # dumping the trained gaussian model
        picklefile = path.split("_")[0]+".gmm"
        cPickle.dump(gmm,open(dest + picklefile,'wb'))
        print ('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)   
        features = np.asarray(())
        count = 0
    count = count + 1

Akshay_Kumar/Akshay_Kumar-1.wav
Akshay_Kumar/Akshay_Kumar-2.wav
Akshay_Kumar/Akshay_Kumar-3.wav
Akshay_Kumar/Akshay_Kumar-4.wav
Akshay_Kumar/Akshay_Kumar-5.wav
Akshay_Kumar/Akshay_Kumar-6.wav
Akshay_Kumar/Akshay_Kumar-7.wav
Akshay_Kumar/Akshay_Kumar-8.wav
Akshay_Kumar/Akshay_Kumar-9.wav
Akshay_Kumar/Akshay_Kumar-10.wav
+ modeling completed for speaker: Akshay.gmm  with data point =  (5694, 40)
Irfan_khan/Irfan_khan-1.wav
Irfan_khan/Irfan_khan-2.wav
Irfan_khan/Irfan_khan-3.wav
Irfan_khan/Irfan_khan-4.wav
Irfan_khan/Irfan_khan-5.wav
Irfan_khan/Irfan_khan-6.wav
Irfan_khan/Irfan_khan-7.wav
Irfan_khan/Irfan_khan-8.wav
Irfan_khan/Irfan_khan-9.wav
Irfan_khan/Irfan_khan-10.wav
+ modeling completed for speaker: Irfan.gmm  with data point =  (5862, 40)
John_Abraham/John_Abraham-1.wav
John_Abraham/John_Abraham-2.wav
John_Abraham/John_Abraham-3.wav
John_Abraham/John_Abraham-4.wav
John_Abraham/John_Abraham-5.wav
John_Abraham/John_Abraham-6.wav
John_Abraham/John_Abraham-7.wav
John_Abraham/John_Abrah

IsADirectoryError: [Errno 21] Is a directory: '/kaggle/working/Speaker-Recognition-GMM/trainingData/'

In [8]:
import os
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
#from featureextraction import extract_features
import warnings
warnings.filterwarnings("ignore")
import time

#path to get test data files name
test_file = "/kaggle/working/Speaker-Recognition-GMM/testDataPath.txt"       
file_paths = open(test_file,'r')

#path to read test data wav files
source   = "/kaggle/working/Speaker-Recognition-GMM/testData/"   

#path where training speakers GMM modles are saved
modelpath =  "/kaggle/working/Speaker-Recognition-GMM/speakerTrainedModelsGMM/"
##"Trained_Speech_Models/"

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the GMM Gaussian Models
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]

error = 0
total_sample = 0.0

print("Press '1' for checking a single Audio or Press '0' for testing a complete set of audio with Accuracy?")
take=int(input().strip())
if take == 1:
    print ("Enter the File name from the sample with .wav notation :")
    path =input().strip()
    print (("Testing Audio : ",path))
    sr,audio = read(source + path)
    vector   = extract_features(audio,sr)
    
    log_likelihood = np.zeros(len(models)) 
    
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    
    winner = np.argmax(log_likelihood)
    print ("\tThe person in the given audio sample is detected as - ", speakers[winner])

    time.sleep(1.0)
elif take == 0:
    test_file = "/kaggle/working/Speaker-Recognition-GMM/testDataPath.txt"         
    file_paths = open(test_file,'r')
    # Read the test directory and get the list of test audio files 
    for path in file_paths:   
        total_sample+= 1.0
        path=path.strip()
        print("Testing Audio : ", path)
        sr,audio = read(source + path)
        vector   = extract_features(audio,sr)
        log_likelihood = np.zeros(len(models)) 
        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()
        winner=np.argmax(log_likelihood)
        print ("\tdetected as - ", speakers[winner])
        checker_name = path.split("_")[0]
        if speakers[winner] != checker_name:
            error += 1
        time.sleep(1.0)
    print (error, total_sample)
    accuracy = ((total_sample - error) / total_sample) * 100

    print ("The Accuracy Percentage for the current testing Performance with MFCC + GMM is : ", accuracy, "%")


print ("Speaker Identified Successfully")

Press '1' for checking a single Audio or Press '0' for testing a complete set of audio with Accuracy?


 1


Enter the File name from the sample with .wav notation :


 Irfak-11.wav


('Testing Audio : ', 'Irfak-11.wav')
	The person in the given audio sample is detected as -  Irfan
Speaker Identified Successfully
